In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df =  pd.read_csv('train.csv', nrows = 1_500_000, parse_dates=["pickup_datetime"], index_col = ['key'])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df.passenger_count >= 7]

In [ ]:
df.fare_amount = abs(df.fare_amount)
df

In [ ]:
df = df[df.fare_amount > 0]
df = df[df.passenger_count <= 10]
df

In [ ]:
df = df.dropna(how = 'any', axis = 'rows')

In [ ]:
df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['hour'] =df['pickup_datetime'].dt.hour
df['day'] = df['pickup_datetime'].dt.day
#!pip install holidays
from holidays import US
holidays_us = US()
df['is_holiday'] = df['pickup_datetime'].dt.date.apply(lambda x: x in holidays_us).astype(int)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # Суббота и воскресенье
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['is_rushhour'] = df['hour'].isin([8, 18, 20, 19, 17, 21]).astype(int) 
df['time_period'] = pd.cut(df['hour'], 
                          bins=[-1, 6, 12, 18, 23],
                          labels=[0, 1, 2, 3]) #вечер день
df['season'] = pd.cut(df['month'],
                     bins=[0, 3, 6, 9, 12],
                     labels=[4,1, 2, 3])
df.time_period = df.time_period.astype(int)
df.season = df.season.astype(int)

In [ ]:
df

In [ ]:
def create_density_heatmap(df):
    density_maps = {}
    for hour in range(24):
        hour_data = df[df['hour'] == hour]
        heatmap, _, _ = np.histogram2d(
            hour_data['pickup_latitude'], 
            hour_data['pickup_longitude'], 
        )
        density_maps[hour] = heatmap
    return density_maps
def plot_density_heatmap(df, figsize=(15, 10)):
    density_maps = create_density_heatmap(df)
    fig, axes = plt.subplots(4, 6, figsize=figsize)
    axes = axes.ravel()
    for hour, heatmap in density_maps.items():
        ax = axes[hour]
        sns.heatmap(heatmap, ax=ax, cmap='YlOrRd')
        ax.set_title(f'Hour {hour}')
    plt.tight_layout()
    return fig
fig = plot_density_heatmap(df)
plt.show()

In [ ]:
pip install reverse_geocoder

In [ ]:
pip install geopy

In [ ]:
import reverse_geocoder as rg
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
NYC_CENTER = (40.7128, -74.0060)  # Manhattan center
JFK_AIRPORT = (40.6413, -73.7781)
LGA_AIRPORT = (40.7769, -73.8740)
EWR_AIRPORT = (40.6895, -74.1745)
def create_distance_features(df):

    df['pickup_dist_to_center'] = df.apply(lambda row: 
        geodesic((row['pickup_latitude'], row['pickup_longitude']), NYC_CENTER).miles, axis=1)
    df['pickup_dist_to_jfk'] = df.apply(lambda row: 
        geodesic((row['pickup_latitude'], row['pickup_longitude']), JFK_AIRPORT).miles, axis=1)
    df['pickup_dist_to_lga'] = df.apply(lambda row: 
        geodesic((row['pickup_latitude'], row['pickup_longitude']), LGA_AIRPORT).miles, axis=1)
    df['pickup_dist_to_ewr'] = df.apply(lambda row: 
        geodesic((row['pickup_latitude'], row['pickup_longitude']), EWR_AIRPORT).miles, axis=1)
    # to EWR
   
    df['dropoff_dist_to_center'] = df.apply(lambda row: 
        geodesic((row['dropoff_latitude'], row['dropoff_longitude']), NYC_CENTER).miles, axis=1)
    df['dropoff_dist_to_jfk'] = df.apply(lambda row: 
        geodesic((row['dropoff_latitude'], row['dropoff_longitude']), JFK_AIRPORT).miles, axis=1)
    df['dropoff_dist_to_lga'] = df.apply(lambda row: 
        geodesic((row['dropoff_latitude'], row['dropoff_longitude']), LGA_AIRPORT).miles, axis=1)
    df['dropoff_dist_to_ewr'] = df.apply(lambda row: 
        geodesic((row['dropoff_latitude'], row['dropoff_longitude']), EWR_AIRPORT).miles, axis=1)
    
    df['trip_distance'] = df.apply(lambda row: 
        geodesic(
            (row['pickup_latitude'], row['pickup_longitude']),
            (row['dropoff_latitude'], row['dropoff_longitude'])
        ).miles, axis=1)
    return df

килерфичи

In [ ]:
def encode_administrative_areas(df):
    # Reverse geocoding
    coordinates = df[['latitude', 'longitude']].values
    results = rg.search(coordinates)
    
    df['admin_area'] = [result['admin1'] for result in results]
    df['city'] = [result['name'] for result in results]
    return df

In [ ]:
def transform_to_metric(df):
    # to UTM coordinates (meters)
    transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32618', always_xy=True)
    
    meters_east, meters_north = transformer.transform(
        df['longitude'].values, 
        df['latitude'].values
    )
    df['meters_east'] = meters_east
    df['meters_north'] = meters_north
    return df

In [ ]:
''' если много неправильных а так дроп
    df[lat_col] = df[lat_col] % 180
    df.loc[df[lat_col] > 90, lat_col] = 180 - df.loc[df[lat_col] > 90, lat_col]
    df.loc[df[lat_col] < -90, lat_col] = -180 - df.loc[df[lat_col] < -90, lat_col]
    df[lon_col] = ((df[lon_col] + 180) % 360) - 180
'''

In [ ]:
def filter_nyc_coordinates(df, lat_col='latitude', lon_col='longitude'):
    """
    Filter coordinates to NYC area
    NYC Bounding Box:
    Latitude: 40.477399 to 40.917577
    Longitude: -74.259090 to -73.700272
    """
    nyc_mask = (
        (df[lat_col] >= 40.477399) & 
        (df[lat_col] <= 40.917577) & 
        (df[lon_col] >= -74.259090) & 
        (df[lon_col] <= -73.700272)
    )
    
    return df[nyc_mask]

In [ ]:
def create_h3_feature(df, lat_col, lon_col, resolution=7):
    """ 
    resolution: H3 resolution (0-15), default 7
    Чем выше resolution, тем более детальное разбиение пространства
    7-8 для городского масштаба
5-6 для регионов
3-4 страны
    """
    df['h3_index'] = df.apply(
        lambda row: h3.geo_to_h3(row[lat_col], row[lon_col], resolution), 
        axis=1
    )
    
    # Создаем one-hot encoding
    h3_encoded = pd.get_dummies(df['h3_index'], prefix='h3_zone')
    
    # Объединяем с исходным датафреймом
    result_df = pd.concat([df, h3_encoded], axis=1)
    
    # Удаляем промежуточный столбец с H3 индексами
    result_df.drop('h3_index', axis=1, inplace=True)
    
    return result_df

In [ ]:
import requests
import rasterio
from rasterio.warp import transform
import ee
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class TerrainEnrichment:
    def __init__(self):
        # Инициализация Google Earth Engine
        ee.Initialize()
        
    def get_elevation(self, df, lat_col='latitude', lon_col='longitude'):
        """
        Получает высоту над уровнем моря используя Open-Elevation API
        """
        elevations = []
        
        for idx, row in df.iterrows():
            url = f"https://api.open-elevation.com/api/v1/lookup?locations={row[lat_col]},{row[lon_col]}"
            response = requests.get(url).json()
            elevation = response['results'][0]['elevation']
            elevations.append(elevation)
            
        df['elevation'] = elevations
        
        # Добавляем производные признаки
        df['elevation_category'] = pd.cut(df['elevation'],
                                        bins=[-np.inf, 0, 200, 500, 1000, np.inf],
                                        labels=['sea_level', 'lowland', 'upland', 'highland', 'mountain'])
        
        return df
    
    def get_landcover(self, df, lat_col='latitude', lon_col='longitude'):
        """
        Получает тип земной поверхности используя Google Earth Engine
        """
        # Загружаем датасет землепользования ESA
        dataset = ee.ImageCollection('ESA/WorldCover/v100').first()
        
        landcover_types = []
        urban_density = []
        
        for idx, row in df.iterrows():
            point = ee.Geometry.Point([row[lon_col], row[lat_col]])
            
            # Получаем тип поверхности
            landcover = dataset.sample(point, 30).first().get('Map').getInfo()
            landcover_types.append(self._decode_landcover(landcover))
            
            # Рассчитываем плотность городской застройки в радиусе 1км
            buffer = point.buffer(1000)
            urban_pixels = dataset.select('Map').eq(50).reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=buffer,
                scale=30
            ).get('Map').getInfo()
            
            urban_density.append(urban_pixels)
        
        df['landcover_type'] = landcover_types
        df['urban_density'] = urban_density
        
        # Добавляем признак природной зоны
        df['nature_zone'] = self._calculate_nature_zone(df[lat_col])
        
        return df
    
    def _decode_landcover(self, code):
        """
        Декодирует коды типов поверхности ESA WorldCover
        """
        landcover_dict = {
            10: 'tree_cover',
            20: 'shrubland',
            30: 'grassland',
            40: 'cropland',
            50: 'built_up',
            60: 'bare_ground',
            70: 'snow_ice',
            80: 'water',
            90: 'wetland'
        }
        return landcover_dict.get(code, 'unknown')
    
    def _calculate_nature_zone(self, latitude):
        """
        Определяет природную зону на основе широты
        """
        zones = pd.cut(abs(latitude),
                      bins=[0, 23.5, 35, 55, 66.5, 90],
                      labels=['tropical', 'subtropical', 'temperate', 'subarctic', 'arctic'])
        return zones
        
# Пример использования
data = {
        'latitude': [55.7558, 55.7522, 55.7539],
        'longitude': [37.6173, 37.6156, 37.6208]
}
df = pd.DataFrame(data)
enricher = TerrainEnrichment()
df = enricher.get_elevation(df)
df = enricher.get_landcover(df)
    



In [ ]:
df = df[df['pickup_latitude'] <= 90]
df = df[df['dropoff_latitude'] <= 90]

In [ ]:
df = df[df['pickup_latitude'] >= -90]
df = df[df['dropoff_latitude'] >= -90]

In [ ]:
df = create_distance_features(df)

In [ ]:
df

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

plt.figure(figsize=(10,10))

cmap = LinearSegmentedColormap.from_list(name='name', colors=['green','yellow','red'])

f, ax = plt.subplots()
points = ax.scatter(df['pickup_longitude'], df['pickup_latitude'], c=df['fare_amount'],
                     cmap=cmap)
f.colorbar(points)

In [ ]:
plt.figure(figsize=(10,10))

cmap = LinearSegmentedColormap.from_list(name='name', colors=['green','yellow','red'])

f, ax = plt.subplots()
points = ax.scatter(df['dropoff_longitude'], df['dropoff_latitude'], c=df['fare_amount'], cmap=cmap)
f.colorbar(points)

In [ ]:
def create_vector_features(df):
    pickup_lat_rad = np.radians(df['pickup_latitude'])
    pickup_lon_rad = np.radians(df['pickup_longitude'])
    dropoff_lat_rad = np.radians(df['dropoff_latitude'])
    dropoff_lon_rad = np.radians(df['dropoff_longitude'])
    
    y = np.sin(dropoff_lon_rad - pickup_lon_rad) * np.cos(dropoff_lat_rad)
    x = np.cos(pickup_lat_rad) * np.sin(dropoff_lat_rad) - \
        np.sin(pickup_lat_rad) * np.cos(dropoff_lat_rad) * np.cos(dropoff_lon_rad - pickup_lon_rad)
    bearing = np.arctan2(y, x)
    
    df['travel_bearing'] = np.degrees(bearing) 
    df['travel_bearing_sin'] = np.sin(bearing)
    df['travel_bearing_cos'] = np.cos(bearing)
    
    df['travel_vector_magnitude'] = np.sqrt(
        (df['dropoff_latitude'] - df['pickup_latitude'])**2 + 
        (df['dropoff_longitude'] - df['pickup_longitude'])**2
    )
    
    df['travel_ns_component'] = df['dropoff_latitude'] - df['pickup_latitude']
    df['travel_ew_component'] = df['dropoff_longitude'] - df['pickup_longitude']

    return df

In [ ]:
df = create_vector_features(df)

In [ ]:
'''def create_dbscan_clusters(df, eps=0.3, min_samples=5):
    coords = df[['pickup_latitude', 'pickup_longitude']].values
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    df['cluster_id'] = clustering.labels_
    return df
# Кластеризация K-средних
kmeans = KMeans(n_clusters=2)
data['cluster'] = kmeans.fit_predict(data[['latitude', 'longitude']])'''

In [ ]:
#df = create_dbscan_clusters(df) - memory limit

In [ ]:
plt.figure(figsize=(20,10))
df['fare_amount'].plot()
plt.xlabel('Date')
plt.ylabel('fare_amount')
plt.show()

In [ ]:
df

In [ ]:
test = pd.read_csv('test (2).csv', parse_dates=["pickup_datetime"],index_col = ['key'])
test.describe()

In [ ]:
df.index = pd.to_datetime(df.index)
test.index = pd.to_datetime(test.index)

In [ ]:
len(set(test.index.unique()) - set(df.index.unique()))

In [ ]:
df.index = pd.to_datetime(df.index)
monthly_df = pd.DataFrame()
monthly_df['fare_amount'] = df['fare_amount'].resample('MS').mean()
plt.plot(monthly_df.index, monthly_df.fare_amount, linewidth=3)

In [ ]:
pip install statsmodels

In [ ]:
from pylab import rcParams
import statsmodels.api as sm
rcParams['figure.figsize'] = 18, 8

decomposition = sm.tsa.seasonal_decompose(monthly_df['fare_amount'], model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
df.index

In [ ]:
df = df.sort_index()
test = test.sort_index()

In [ ]:
'''df['rolling_mean_7d'] = df['fare_amount'].rolling('7D').mean()
df['rolling_mean_24h'] = df['fare_amount'].rolling('24H').mean()

df['rolling_sum_7d'] = df['fare_amount'].rolling('7D').sum()
df['rolling_sum_24h'] = df['fare_amount'].rolling('24H').sum()

df['rolling_max_7d'] = df['fare_amount'].rolling('7D').max()
df['rolling_max_24h'] = df['fare_amount'].rolling('24H').max()

df['rolling_min_7d'] = df['fare_amount'].rolling('7D').min()
df['rolling_min_24h'] = df['fare_amount'].rolling('24H').min()'''

In [ ]:
df.index.unique().max(), test.index.unique().max()

In [ ]:
test

In [ ]:
test['year'] = test['pickup_datetime'].dt.year
test['month'] = test['pickup_datetime'].dt.month
test['day_of_week'] = test['pickup_datetime'].dt.dayofweek
test['hour'] =test['pickup_datetime'].dt.hour
test['day'] = test['pickup_datetime'].dt.day
test['is_holiday'] = test['pickup_datetime'].dt.date.apply(lambda x: x in holidays_us).astype(int)
test['is_weekend'] = test['day_of_week'].isin([5, 6]).astype(int)  # Суббота и воскресенье
test['hour_sin'] = np.sin(2 * np.pi * test['hour'] / 24)
test['hour_cos'] = np.cos(2 * np.pi * test['hour'] / 24)
test['month_sin'] = np.sin(2 * np.pi * test['month'] / 12)
test['month_cos'] = np.cos(2 * np.pi * test['month'] / 12)
test['is_rushhour'] = test['hour'].isin([8, 18, 20, 19, 17, 21]).astype(int) 
test['time_period'] = pd.cut(test['hour'], 
                          bins=[-1, 6, 12, 18, 23],
                          labels=[0, 1, 2, 3]) #вечер день
test['season'] = pd.cut(test['month'],
                     bins=[0, 3, 6, 9, 12],
                     labels=[4,1, 2, 3])
test.time_period = test.time_period.astype(int)
test.season = test.season.astype(int)
test = create_distance_features(test)
test = create_vector_features(test)

In [ ]:
df.shape, test.shape

In [ ]:
df.index.unique().shape

In [ ]:
df[df.index.duplicated()]

In [ ]:
test[test.index.duplicated()]

In [ ]:
test = test.reset_index()

In [ ]:
df = df.reset_index()

In [ ]:
def calculate_rolling_means(train_df, test_df, time_column='time_key', value_column='fare_amount'):

    train_df['is_train'] = True
    test_df['is_train'] = False
    
    combined_df = pd.concat([train_df, test_df], axis=0)
    combined_df = combined_df.sort_values(time_column)

    three_hours = pd.Timedelta(hours=3)
    three_days = pd.Timedelta(days=3)

    train_data = combined_df[combined_df['is_train']][[time_column, value_column]]
    combined_df['rolling_mean_3h'] = train_data.rolling(
        window=three_hours,
        min_periods=1,
        closed='left',
        on=time_column
    ).mean()['fare_amount']
    
    combined_df['rolling_mean_3d'] = train_data.rolling(
        window=three_days,
        min_periods=1,
        closed='left',
        on=time_column
    ).mean()['fare_amount']
    
    train_processed = combined_df[combined_df['is_train']].drop('is_train', axis=1)
    test_processed = combined_df[~combined_df['is_train']].drop('is_train', axis=1)
    
    return train_processed, test_processed

In [ ]:
train, test = calculate_rolling_means(df,test)

In [ ]:
train.shape, test.shape

In [ ]:
test.drop(columns = ['fare_amount'],inplace = True)

In [ ]:
test.isna().sum()

In [ ]:
train.rolling_mean_3h.mean()

In [ ]:
train['rolling_mean_3h'] = train['rolling_mean_3h'].fillna(train.rolling_mean_3h.mean())
#test['rolling_mean_3d'] = train['rolling_mean_3d'].fillna(9.695963242861593)
test['rolling_mean_3h'] = test['rolling_mean_3h'].fillna(9.72152837095675)
test['rolling_mean_3d'] = test['rolling_mean_3d'].fillna(9.695963242861593)

In [ ]:
from statsmodels.tsa.seasonal import STL

def extract_time_components(train_df, test_df, time_column='time_key', target_column='fare_amount'):
    train_ts = train_df.set_index(time_column).resample('H')[target_column].mean().fillna(method='ffill')
    stl = STL(train_ts, period=24)
    result = stl.fit()
    
    train_components = pd.DataFrame(index=train_ts.index)
    train_components['trend'] = result.trend
    train_components['seasonal'] = result.seasonal
    
    seasonal_pattern = result.seasonal.values[:24]
    train_df['seasonal'] = train_df['hour'].map(dict(enumerate(seasonal_pattern)))
    test_df['seasonal'] = test_df['hour'].map(dict(enumerate(seasonal_pattern)))
    
    return train_df, test_df


In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
train,test = extract_time_components(train,test)

In [ ]:
train.shape, test.shape

In [ ]:
train.to_csv('df_clear.csv')
test.to_csv('test_clear.csv')

In [ ]:
corr = train.corr()
corr['fare_amount']

In [ ]:
#from sklearn.metrics import mean_squared_error
#!pip install catboost
from catboost import CatBoostRegressor

In [ ]:
import gc
gc.collect()

In [ ]:
model = CatBoostRegressor()
x = train.drop(columns = ['fare_amount', 'time_key', 'rolling_mean_3d', 'pickup_datetime'])
y = train['fare_amount']
model.fit(x,y)

In [ ]:
test.drop(columns = ['rolling_mean_3d'],inplace = True) #"['pickup_datetime', 'time_key'] not found in axis"

In [ ]:
subm = pd.read_csv('sample_submission (2).csv')

In [ ]:
subm

In [ ]:
subm['fare_amount'] = model.predict(test)

In [ ]:
subm.to_csv('subm2.csv',sep = ',',index = False)

In [ ]:
#mda catboost podkachal nu nichego, best score without rolling and seasonality

In [ ]:
train.corr()

In [ ]:
train1 = train.drop(columns = ['rolling_mean_3h' , 'rolling_mean_3d'])

In [ ]:
test1 = test.drop(columns = ['rolling_mean_3h'])
test1.shape

In [ ]:
test1 = test.copy()

In [ ]:
test1.shape

In [ ]:
from gluonts.mx import SimpleFeedForwardEstimator, Trainer
estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    trainer=Trainer(ctx="cpu", epochs=5, learning_rate=1e-3, num_batches_per_epoch=100),
 #https://ts.gluon.ai/stable/tutorials/forecasting/quick_start_tutorial.html

ПРОВЕРИМ НА СТАЦИОНАРНОСТЬ

In [ ]:
train1 = train1.set_index('time_key')
train1.shape

In [ ]:
test1

In [ ]:
train1.index = pd.to_datetime(train1.index)
monthly_OS = pd.DataFrame()
monthly_OS['fare_amount'] = train1['fare_amount'].resample('MS').mean()
from statsmodels.tsa.stattools import adfuller
print ('Results of Dickey-Fuller Test:')
dftest = adfuller(monthly_OS, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print (dfoutput)

ужасно нестационарные данные, p - value должен быть 5%

In [ ]:
do = train1['fare_amount']
do = do.reset_index()
do

In [ ]:
do = do.groupby('time_key')['fare_amount'].sum().reset_index()
do = do.set_index('time_key')
do.index = pd.to_datetime(do.index)
yy = do['fare_amount'].resample('MS').mean()
ts_log = np.log(yy)
plt.plot(ts_log)

In [ ]:
#train1['fare_amount'] = np.log(train1['fare_amount'])
train1.describe()

### Полезные ссылки
- https://www.kaggle.com/code/kanncaa1/time-series-prediction-tutorial-with-eda - ARIMA
- https://www.kaggle.com/code/breemen/nyc-taxi-fare-data-exploration - krutoi chel
- 